In [10]:
import os
from openai import OpenAI
from PIL import Image
import base64
import json
import pandas as pd
import numpy as np
from collections import Counter
import datetime
import pymssql
from threading import Thread
import functools
import time

os.environ["OPENAI_API_KEY"] = "Key_Value"
conn = pymssql.connect(host=r"(local)", database='WorkDB4', charset='utf8')

def most_frequent(numbers):
    # Count the frequency of each number in the list
    frequency = Counter(numbers)
    # Find the number with the highest frequency
    most_common = frequency.most_common(1)
    return most_common[0][0] if most_common else None

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

In [11]:
question='''58세 남자가 30분 전에 디클로페낙 주사를 맞고 피부에 발진이 생겼다. 가려움증은 있었으나, 호흡곤란이나 복통은 없었다. 2년 전부터 고혈압으로 로살탄을 복용하였으며, 1년 전부터는 당뇨병과 통풍으로 메트포르민과 알로퓨리놀을 추가 복용하였다. 1주일 전부터 봉와직염으로 세파클로르를 복용 중이다. 혈압 130/60 mmHg, 맥박 80회/분, 호흡 16회/분, 체온 38.2도였다. 피부사진(사진18)이다. 원인약제는?
1. 로살탄
2. 메트포르민
3. 세파클로르
4. 디클로페낙
5. 알로퓨리놀'''


image=encode_image('images/sample_image1.PNG')


system_content='''You are a Korean medical school student taking the Korean Medical Licensing Examinations to become a Korean doctor. 
Answer the questions. Give the answer in the following JSON format
{
  "reasoning": "(elaboarate your reasonining here)",
  "answer": 4
}

'''

client = OpenAI()

df=pd.DataFrame(columns=['Question','Image','Answer'])
element=[question,image,4]
for i in range(10):
    df.loc[len(df)]=element

In [12]:
df

,Question,Image,Answer
0,58세 남자가 30분 전에 디클로페낙 주사를 맞고 피부에 발진이 생겼다. 가려움증은...,iVBORw0KGgoAAAANSUhEUgAAAZ8AAAHwCAYAAAB5QGIbAA...,4
1,58세 남자가 30분 전에 디클로페낙 주사를 맞고 피부에 발진이 생겼다. 가려움증은...,iVBORw0KGgoAAAANSUhEUgAAAZ8AAAHwCAYAAAB5QGIbAA...,4
2,58세 남자가 30분 전에 디클로페낙 주사를 맞고 피부에 발진이 생겼다. 가려움증은...,iVBORw0KGgoAAAANSUhEUgAAAZ8AAAHwCAYAAAB5QGIbAA...,4
3,58세 남자가 30분 전에 디클로페낙 주사를 맞고 피부에 발진이 생겼다. 가려움증은...,iVBORw0KGgoAAAANSUhEUgAAAZ8AAAHwCAYAAAB5QGIbAA...,4
4,58세 남자가 30분 전에 디클로페낙 주사를 맞고 피부에 발진이 생겼다. 가려움증은...,iVBORw0KGgoAAAANSUhEUgAAAZ8AAAHwCAYAAAB5QGIbAA...,4
5,58세 남자가 30분 전에 디클로페낙 주사를 맞고 피부에 발진이 생겼다. 가려움증은...,iVBORw0KGgoAAAANSUhEUgAAAZ8AAAHwCAYAAAB5QGIbAA...,4
6,58세 남자가 30분 전에 디클로페낙 주사를 맞고 피부에 발진이 생겼다. 가려움증은...,iVBORw0KGgoAAAANSUhEUgAAAZ8AAAHwCAYAAAB5QGIbAA...,4
7,58세 남자가 30분 전에 디클로페낙 주사를 맞고 피부에 발진이 생겼다. 가려움증은...,iVBORw0KGgoAAAANSUhEUgAAAZ8AAAHwCAYAAAB5QGIbAA...,4
8,58세 남자가 30분 전에 디클로페낙 주사를 맞고 피부에 발진이 생겼다. 가려움증은...,iVBORw0KGgoAAAANSUhEUgAAAZ8AAAHwCAYAAAB5QGIbAA...,4
9,58세 남자가 30분 전에 디클로페낙 주사를 맞고 피부에 발진이 생겼다. 가려움증은...,iVBORw0KGgoAAAANSUhEUgAAAZ8AAAHwCAYAAAB5QGIbAA...,4


In [4]:
def prompt_engineering_eval(df, system_content, model_ver, numbers, temperature_setting):
    time1=datetime.datetime.now()
    true_num=0
    false_num=0
    for i in range(len(df)):
        question=df['Question'][i]
        image=df['Image'][i]
        answer=df['Answer'][i]
        
        response = client.chat.completions.create(
            model=model_ver,
            n=numbers,
            temperature=temperature_setting,
            response_format={ "type": "json_object" },
            messages=[
                {"role": "system", "content": system_content},
                {"role": "user", "content": [
                    {"type": "text", "text": question},
                    {"type": "image_url", "image_url": {
                        "url": f"data:image/png;base64,{image}"}
                    }

                ]}
            ],
        )
        
        answer_list=[]
        for j in range(numbers):
            try:
                data_dict = json.loads(response.choices[j].message.content)
                data_dict['reasoning']
                answer_list.append(int(data_dict['answer']))
            except:
                pass
        answer_frequent = most_frequent(answer_list)
        if answer==answer_frequent:
            true_num+=1
        else:
            false_num+=1
    time2=datetime.datetime.now()
    print(time2-time1)
    return true_num, false_num

In [5]:
prompt_engineering_eval(df, system_content, 'gpt-4o', 10, 1.0)

0:01:01.485795


(10, 0)

In [13]:
def timeout(timeout):
    def deco(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            res = [Exception('function [%s] timeout [%s seconds] exceeded!' % (func.__name__, timeout))]
            def newFunc():
                try:
                    res[0] = func(*args, **kwargs)
                except Exception as e:
                    res[0] = e
            t = Thread(target=newFunc)
            t.daemon = True
            try:
                t.start()
                t.join(timeout)
            except Exception as je:
                print ('error starting thread')
                raise je
            ret = res[0]
            #if isinstance(ret, BaseException):
            #    raise ret
            return ret
        return wrapper
    return deco

In [14]:
newtablename='20240622_test'
sql_createtable="CREATE TABLE [" + newtablename +"""] 
(
    questionnum    NVARCHAR(20),
    GPT_reasoning    NVARCHAR(max),
    GPT_answer    NVARCHAR(max)
)

"""
conn = pymssql.connect(host=r"(local)", database='WorkDB4', charset='utf8')
with conn:
    with conn.cursor() as cur:
        cur.execute(sql_createtable)
        conn.commit()
 

In [15]:
@timeout(1.2)
def insert_into_sql(i, system_content, model_ver, numbers, temperature_setting):

    question=df['Question'][i]
    image=df['Image'][i]
    answer=df['Answer'][i]
        
    response = client.chat.completions.create(
        model=model_ver,
        n=numbers,
        temperature=temperature_setting,
        response_format={ "type": "json_object" },
        messages=[
            {"role": "system", "content": system_content},
            {"role": "user", "content": [
                {"type": "text", "text": question},
                {"type": "image_url", "image_url": {
                    "url": f"data:image/png;base64,{image}"}
                }

            ]}
        ],
    )
    
    aa2=response.choices
    
    for j in range(len(aa2)):
        data_dict = json.loads(aa2[j].message.content)
        try:
            reasoning_temp=data_dict['reasoning']
            answer_temp=data_dict['answer']
            
            conn = pymssql.connect(host=r"(local)", database='WorkDB4', charset='utf8')
            sql = "INSERT INTO [" + newtablename+"] (questionnum, GPT_reasoning, GPT_answer) VALUES (%s, %s, %s)"

            with conn:
                with conn.cursor() as cur:
                    cur.execute(sql, (str(i),str(reasoning_temp),str(answer_temp)))
                    conn.commit()
        except:
            pass


In [16]:
for i in range(len(df)):
    insert_into_sql(i, system_content, 'gpt-4o', 10, 1.0)